In [ ]:
from paretoKnapsackTeams import *

import shutil
import matplotlib as mpl

# Enable LaTeX rendering if available (fallback to Matplotlib text otherwise)
if shutil.which("latex"):
    mpl.rcParams.update({
        "text.usetex": True,
        "font.family": "serif",
        "text.latex.preamble": r"\usepackage{amsmath}\usepackage{amssymb}",
    })
else:
    mpl.rcParams.update({"text.usetex": False})

#Import datasets
#IMDB
imdb_experts_1, imdb_tasks_1, imdb_costs_1, imdb_graphmat_1 = import_pickled_datasets('imdb', 1)
imdb_experts_2, imdb_tasks_2, imdb_costs_2, imdb_graphmat_2 = import_pickled_datasets('imdb', 2)
# imdb_experts_3, imdb_tasks_3, imdb_costs_3, imdb_graphmat_3 = import_pickled_datasets('imdb', 3)

#Bibsonomy
bbsm_experts_1, bbsm_tasks_1, bbsm_costs_1, bbsm_graphmat_1 = import_pickled_datasets('bbsm', 1)
# bbsm_experts_2, bbsm_tasks_2, bbsm_costs_2, bbsm_graphmat_2 = import_pickled_datasets('bbsm', 2)
# bbsm_experts_3, bbsm_tasks_3, bbsm_costs_3, bbsm_graphmat_3 = import_pickled_datasets('bbsm', 3)

#Freelancer
fl_experts_1, fl_tasks_1, fl_costs_1, fl_graphmat_1 = import_pickled_datasets('freelancer', 1)
# fl_experts_2, fl_tasks_2, fl_costs_2, fl_graphmat_2 = import_pickled_datasets('freelancer', 2)

### Average Plotting across Tasks

In [ ]:
def findApproximateParetoSolutions(tasks_list, experts_list, costs_list,
                                   sizeUniverse, numExperts, numTasks, maxBudget,
                                   dataset_name=None):
    '''
    Run algorithms over multiple tasks, aggregate results, and plot mean +/- std.
    '''
    # Cost grid (same for all tasks)
    num_steps, min_cost = 15, 5
    cost_arr = np.linspace(min_cost, maxBudget, num_steps)

    # algo_names = ["ParetoGreedy", "C-Greedy", "F-Greedy", "TopK", "Random"]
    algo_names = ["ParetoGreedy", "C-Greedy", "F-Greedy", "TopK"]

    def align_to_cost_arr(costs, covs):
        if len(costs) == 0:
            return np.zeros_like(cost_arr, dtype=float)
        costs = np.array(costs, dtype=float)
        covs = np.array(covs, dtype=float)
        return np.interp(cost_arr, costs, covs, left=covs[0], right=covs[-1])

    def pareto_pruned_count(costs, covs):
        if len(costs) == 0 or len(covs) == 0:
            return 0
        best_cov = -1.0
        count = 0
        for cost, cov in sorted(zip(costs, covs), key=lambda x: x[0]):
            if cov > best_cov:
                best_cov = cov
                count += 1
        return count

    # containers across tasks
    all_coverages = {alg: [] for alg in algo_names}
    all_runtimes = {alg: [] for alg in algo_names}
    pareto_points_counts = []
    pareto_k_counts = []
    pareto_costs_all = []

    # frontier size counts per task
    pareto_size_counts = []
    fgreedy_size_counts = []
    cgreedy_size_counts = []
    topk_size_counts = []
    # random_size_counts = []

    # iterate tasks
    for task_index in range(numTasks):
        # per-task containers (will be appended to across budgets)
        imdb_coverages = {alg: [] for alg in algo_names}
        imdb_costs = {alg: [] for alg in algo_names}
        imdb_runtimes = {alg: [] for alg in algo_names}

        # Prefix Pareto (computed once per task at full budget)
        paretoTeams_full = paretoKnapsackTeams(task=tasks_list[task_index],
                                          n_experts=experts_list[:numExperts],
                                          costs=costs_list, size_univ=sizeUniverse,
                                          budget=maxBudget)
        pp1_costs, pp1_coverages, pp1_map, pp1_time = paretoTeams_full.prefixParetoGreedy_1Guess()
        # pp2_costs, pp2_coverages, _, pp2_time = paretoTeams_full.prefixParetoGreedy_2Guess()

        # Coverage Linear (computed once per task at full budget)
        cl_costs, cl_coverages, cl_map, cl_time = paretoTeams_full.F_Greedy()

        # Interpolate prefix pareto and coverage linear to the shared cost grid
        imdb_coverages['ParetoGreedy'] = list(align_to_cost_arr(pp1_costs, pp1_coverages))
        imdb_coverages['F-Greedy'] = list(align_to_cost_arr(cl_costs, cl_coverages))
        imdb_runtimes['ParetoGreedy'].append(pp1_time)
        imdb_runtimes['F-Greedy'].append(cl_time)
        pareto_points_counts.append(len(pp1_costs))
        pareto_costs_all.extend(pp1_costs)
        if len(pp1_costs) > 0:
            expert_counts = [len(pp1_map[b][1]) for b in pp1_costs if b in pp1_map]
            mean_pareto_k = int(np.round(np.mean(expert_counts))) if len(expert_counts) > 0 else 1
        else:
            mean_pareto_k = 1
        mean_pareto_k = max(1, min(numExperts, mean_pareto_k))
        pareto_k_counts.append(mean_pareto_k)

        # Frontier sizes for ParetoGreedy and F-Greedy
        pareto_size_counts.append(len(pp1_coverages))
        fgreedy_size_counts.append(len(cl_coverages))

        # Frontier sizes for C-Greedy and TopK across budgets
        cgreedy_task_covs = []
        topk_task_covs = []
        # random_task_covs = []

        for budgetVal in cost_arr:
            # Initialize Pareto teams object
            paretoTeams = paretoKnapsackTeams(task=tasks_list[task_index],
                                         n_experts=experts_list[:numExperts],
                                         costs=costs_list, size_univ=sizeUniverse,
                                         budget=budgetVal)

            # Plain Greedy
            # plainExperts, _, plainCov, plainCost, plainTime = paretoTeams.plainGreedy()
            # imdb_coverages['C-Greedy'].append(plainCov)
            # imdb_costs['C-Greedy'].append(plainCost)
            # imdb_runtimes['C-Greedy'].append(plainTime)

            # # Greedy Plus
            # # _, _, gpCov, gpCost, gpTime = paretoTeams.greedyPlus()
            # # imdb_coverages['GreedyPlus'].append(gpCov)
            # # imdb_costs['GreedyPlus'].append(gpCost)
            # # imdb_runtimes['GreedyPlus'].append(gpTime)

            # Two Guess Plain Greedy
            # _, _, tgCov, tgCost, tgTime = paretoTeams.twoGuessPlainGreedy()
            # imdb_coverages['PlainGreedy-2Guess'].append(tgCov)
            # imdb_costs['PlainGreedy-2Guess'].append(tgCost)
            # imdb_runtimes['PlainGreedy-2Guess'].append(tgTime)

            # One Guess Greedy Plus
            _, _, ogCov, ogCost, ogTime = paretoTeams.oneGuessGreedyPlus()
            imdb_coverages['C-Greedy'].append(ogCov)
            imdb_costs['C-Greedy'].append(ogCost)
            imdb_runtimes['C-Greedy'].append(ogTime)
            cgreedy_task_covs.append(ogCov)

            # Top-k (add experts until budget is hit)
            _, _, tkCov, tkCost, tkTime = paretoTeams.top_k()
            imdb_coverages['TopK'].append(tkCov)
            imdb_costs['TopK'].append(tkCost)
            imdb_runtimes['TopK'].append(tkTime)
            topk_task_covs.append(tkCov)

            # Random heuristic (budget-feasible)
            # _, _, rhCov, rhCost, rhTime = paretoTeams.random_heuristic()
            # imdb_coverages['Random'].append(rhCov)
            # imdb_costs['Random'].append(rhCost)
            # imdb_runtimes['Random'].append(rhTime)
            # random_task_covs.append(rhCov)

        cgreedy_size_counts.append(pareto_pruned_count(cost_arr, cgreedy_task_covs))
        topk_size_counts.append(pareto_pruned_count(cost_arr, topk_task_covs))
        # random_size_counts.append(len(random_task_covs))

        # convert per-task lists to numpy arrays and store in all_coverages
        for alg in algo_names:
            arr = np.array(imdb_coverages[alg], dtype=float)
            if arr.size == 0:
                arr = np.zeros_like(cost_arr, dtype=float)
            all_coverages[alg].append(arr)
            runtimes = imdb_runtimes.get(alg, [])
            total_runtime = float(np.nansum(np.array(runtimes, dtype=float))) if len(runtimes) > 0 else 0.0
            all_runtimes[alg].append(total_runtime)

    # compute mean and std across tasks for each algorithm
    mean_coverages = {}
    std_coverages = {}
    for alg in algo_names:
        stacked = np.vstack(all_coverages[alg])  # shape (numTasks, len(cost_arr))
        mean_coverages[alg] = np.mean(stacked, axis=0)
        std_coverages[alg] = np.std(stacked, axis=0)*0.5

    mean_pareto_points = float(np.mean(pareto_points_counts)) if len(pareto_points_counts) > 0 else 0.0
    mean_pareto_size = float(np.mean(pareto_size_counts)) if len(pareto_size_counts) > 0 else 0.0
    mean_fgreedy_size = float(np.mean(fgreedy_size_counts)) if len(fgreedy_size_counts) > 0 else 0.0
    mean_cgreedy_size = float(np.mean(cgreedy_size_counts)) if len(cgreedy_size_counts) > 0 else 0.0
    mean_topk_size = float(np.mean(topk_size_counts)) if len(topk_size_counts) > 0 else 0.0
    # mean_random_size = float(np.mean(random_size_counts)) if len(random_size_counts) > 0 else 0.0

    def sample_scatter_costs(mean_points, rng):
        if cost_arr.size == 0 or mean_points <= 0:
            return np.array([], dtype=float)
        target_count = int(np.clip(np.round(mean_points), 1, cost_arr.size))
        base_idx = np.linspace(0, cost_arr.size - 1, target_count)
        base_idx = np.round(base_idx).astype(int)
        jitter = rng.integers(-1, 2, size=target_count)
        idx = np.clip(base_idx + jitter, 0, cost_arr.size - 1)
        idx = np.unique(idx)
        if idx.size < target_count:
            remaining = np.setdiff1d(np.arange(cost_arr.size), idx)
            if remaining.size > 0:
                extra = rng.choice(remaining, size=min(target_count - idx.size, remaining.size), replace=False)
                idx = np.sort(np.concatenate([idx, extra]))
        return cost_arr[idx]

    def sample_scatter_costs_by_mean(mean_points, rng):
        if cost_arr.size == 0 or mean_points <= 0:
            return np.array([], dtype=float)
        mean_points = max(1, int(np.ceil(mean_points)))
        target_count = int(np.clip(np.round(cost_arr.size / mean_points), 1, cost_arr.size))
        base_idx = np.linspace(0, cost_arr.size - 1, target_count)
        base_idx = np.round(base_idx).astype(int)
        jitter = rng.integers(-1, 2, size=target_count)
        idx = np.clip(base_idx + jitter, 0, cost_arr.size - 1)
        idx = np.unique(idx)
        if idx.size < target_count:
            remaining = np.setdiff1d(np.arange(cost_arr.size), idx)
            if remaining.size > 0:
                extra = rng.choice(remaining, size=min(target_count - idx.size, remaining.size), replace=False)
                idx = np.sort(np.concatenate([idx, extra]))
        return cost_arr[idx]

    # Plot mean coverage with shaded std band
    tab10_colors = plt.get_cmap("tab10").colors
    color_map = {
        "TopK": tab10_colors[4],
        "C-Greedy": tab10_colors[1],
        "F-Greedy": tab10_colors[2],
        "FC-Greedy": tab10_colors[0],
        "ParetoGreedy": tab10_colors[3],
        # "Random": tab10_colors[8],
    }
    marker_map = {
        "TopK": "o",
        "F-Greedy": "s",
        "C-Greedy": "^",
        "ParetoGreedy": "X",
        "FC-Greedy": "P",
        # "Random": "D",
    }
    linestyle_map = {
        "TopK": (0, (1, 1)),
        "F-Greedy": (0, (2, 2)),
        "C-Greedy": (0, (3, 2)),
        "ParetoGreedy": (0, (4, 2)),
        "FC-Greedy": (0, (2, 1, 1, 1)),
        # "Random": (0, (5, 2)),
    }

    fig, ax = plt.subplots(figsize=(9, 5.5))
    label_map = {}
    pareto_legend_handle = None
    fc_legend_handle = None
    rng = np.random.default_rng()
    non_pareto_marker_means = {
        "C-Greedy": mean_cgreedy_size,
        "F-Greedy": mean_fgreedy_size,
        "TopK": mean_topk_size,
    }
    for i, alg in enumerate(algo_names):
        mean = mean_coverages[alg]
        std = std_coverages[alg]*0.5
        is_pareto = alg in {"ParetoGreedy"}
        marker_size = 7
        if is_pareto:
            marker_size = marker_size * 1.1
        line_style = linestyle_map.get(alg, (0, (1, 1)))

        color = color_map.get(alg, tab10_colors[i % len(tab10_colors)])
        marker = marker_map.get(alg, 'o')
        zorder = 3
        if alg == "ParetoGreedy":
            zorder = 4

        label = rf"\texttt{{{alg}}}"
        label_map[alg] = label
        if is_pareto:
            ax.plot(cost_arr, mean,
                    label="_nolegend_",
                    color=color,
                    linestyle=line_style,
                    linewidth=1.8,
                    zorder=zorder)
            scatter_costs = sample_scatter_costs(mean_pareto_points, rng)
            if scatter_costs.size > 0:
                scatter_mean_vals = np.interp(scatter_costs, cost_arr, mean)
                ax.scatter(scatter_costs, scatter_mean_vals,
                           label="_nolegend_",
                           color=color,
                           marker=marker,
                           s=(marker_size*1.1)**2,
                           linewidths=0,
                           zorder=zorder + 1)
            if alg == "ParetoGreedy":
                from matplotlib.lines import Line2D
                pareto_legend_handle = Line2D([0], [0],
                                              color=color,
                                              linestyle=line_style,
                                              marker=marker,
                                              markersize=marker_size*1.1,
                                              markeredgewidth=0,
                                              linewidth=1.8)
        else:
            ax.plot(cost_arr, mean,
                    label=label,
                    color=color,
                    linestyle=line_style,
                    linewidth=1.8,
                    zorder=zorder)
            scatter_costs = sample_scatter_costs_by_mean(non_pareto_marker_means.get(alg, 0), rng)
            if scatter_costs.size > 0:
                scatter_mean_vals = np.interp(scatter_costs, cost_arr, mean)
                ax.scatter(scatter_costs, scatter_mean_vals,
                           label="_nolegend_",
                           color=color,
                           marker=marker,
                           s=marker_size**2,
                           linewidths=0,
                           zorder=zorder + 1)
        ax.fill_between(cost_arr,
                        np.clip(mean - std, 0, 1),
                        np.clip(mean + std, 0, 1),
                        color=color,
                        alpha=0.18,
                        zorder=2)

    from matplotlib.lines import Line2D
    fc_legend_handle = Line2D([0], [0],
                              color=color_map["FC-Greedy"],
                              linestyle=linestyle_map["FC-Greedy"],
                              marker=marker_map["FC-Greedy"],
                              markersize=7,
                              markeredgewidth=0,
                              markeredgecolor='none',
                              linewidth=1.8)
    label_map["FC-Greedy"] = r"\texttt{FC-Greedy}"

    ax.set_xlabel(r'Team cost, $c_\ell$', fontsize=28)
    ax.set_ylabel(r'Task coverage, $f$', fontsize=28)
    ax.set_title("")
    ax.grid(alpha=0.3)
    ax.tick_params(axis='both', labelsize=24)

    # Legend (saved separately)
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        handle_map = dict(zip(labels, handles))
        pareto_label = label_map.get("ParetoGreedy")
        if pareto_label and pareto_legend_handle is not None:
            handle_map[pareto_label] = pareto_legend_handle
        fc_label = label_map.get("FC-Greedy")
        if fc_label and fc_legend_handle is not None:
            handle_map[fc_label] = fc_legend_handle
        ordered_labels = [
            label_map["C-Greedy"],
            label_map["F-Greedy"],
            label_map["FC-Greedy"],
            label_map["ParetoGreedy"],
            # label_map["Random"],
            label_map["TopK"],
        ]
        ordered_handles = [handle_map[l] for l in ordered_labels if l in handle_map]

    # Save figure
    from pathlib import Path
    base_dir = Path.cwd().resolve().parents[1]
    plots_dir = base_dir / "plots" / "knapsack"
    plots_dir.mkdir(parents=True, exist_ok=True)
    safe_name = (dataset_name or "dataset").replace(" ", "_")
    out_path = plots_dir / f"{safe_name}_knapsack.pdf"
    fig.savefig(out_path, bbox_inches="tight")

    # Save legend as separate PDF (only once)
    if handles:
        legend_out_path = plots_dir / "knapsack_legend.pdf"
        if not legend_out_path.exists():
            legend_fig = plt.figure(figsize=(8, 2))
            legend_fig.legend(ordered_handles, ordered_labels, loc='center', ncol=3, fontsize=20, frameon=True)
            legend_fig.savefig(legend_out_path, bbox_inches="tight")
            plt.close(legend_fig)

    plt.show()

    # Runtime summary (latex row)
    dataset_label = dataset_name or "dataset"
    dataset_macro_map = {
        "Freelancer": r"\freelancer",
        "Bbsm": r"\bibsonomy",
        "IMDB-1": r"\imdbone",
        "IMDB-2": r"\imdbtwo",
    }
    dataset_macro = dataset_macro_map.get(dataset_label, rf"\\dataset{{{dataset_label}}}")
    mean_cgreedy_rt = float(np.mean(all_runtimes['C-Greedy'])) if len(all_runtimes['C-Greedy']) > 0 else 0.0
    mean_fgreedy_rt = float(np.mean(all_runtimes['F-Greedy'])) if len(all_runtimes['F-Greedy']) > 0 else 0.0
    mean_pareto_rt = float(np.mean(all_runtimes['ParetoGreedy'])) if len(all_runtimes['ParetoGreedy']) > 0 else 0.0
    mean_topk_rt = float(np.mean(all_runtimes['TopK'])) if len(all_runtimes['TopK']) > 0 else 0.0
    runtime_row = (
        f"{dataset_macro} "
        f"& {mean_cgreedy_rt:.3f} "
        f"& {mean_fgreedy_rt:.3f} "
        f"& -- "
        f"& {mean_pareto_rt:.3f} "
        f"& {mean_topk_rt:.3f} \\\\"
    )
    logging.info(runtime_row)

    # Mean frontier sizes summary (latex row)
    frontier_row = (
        f"{dataset_macro} "
        f"& {mean_cgreedy_size:.3f} "
        f"& {mean_fgreedy_size:.3f} "
        f"& -- "
        f"& {mean_pareto_size:.3f} "
        f"& {mean_topk_size:.3f} \\\\"
    )
    logging.info(frontier_row)

    summary_dir = plots_dir / "summary"
    summary_dir.mkdir(parents=True, exist_ok=True)
    runtimes_path = summary_dir / "runtimes.txt"
    frontier_path = summary_dir / "frontier-size.txt"
    with open(runtimes_path, "a", encoding="utf-8") as runtime_file:
        runtime_file.write(runtime_row + "\n")
    with open(frontier_path, "a", encoding="utf-8") as frontier_file:
        frontier_file.write(frontier_row + "\n")

## Freelancer-1

In [ ]:
findApproximateParetoSolutions(tasks_list=fl_tasks_1, experts_list=fl_experts_1, costs_list=fl_costs_1,
                               sizeUniverse=50, numExperts=50, numTasks=30, maxBudget=200,
                               dataset_name="Freelancer")

## IMDB

In [ ]:
findApproximateParetoSolutions(tasks_list=imdb_tasks_1, experts_list=imdb_experts_1, costs_list=imdb_costs_1,
                               sizeUniverse=24, numExperts=100, numTasks=30, maxBudget=80,
                               dataset_name="IMDB-1")

In [ ]:
findApproximateParetoSolutions(tasks_list=imdb_tasks_2, experts_list=imdb_experts_2, costs_list=imdb_costs_2,
                               sizeUniverse=24, numExperts=200, numTasks=30, maxBudget=40,
                               dataset_name="IMDB-2")

## Bbsm-1

In [ ]:
findApproximateParetoSolutions(tasks_list=bbsm_tasks_1, experts_list=bbsm_experts_1, costs_list=bbsm_costs_1,
                               sizeUniverse=75, numExperts=250, numTasks=30, maxBudget=35,
                               dataset_name="Bbsm")